In [5]:
import dash
import pandas as pd
import dash_bootstrap_components as dbc
from dash import Dash, dash_table, dcc, html, Input, Output, State
from dash.exceptions import PreventUpdate
import base64
import datetime
import io
import plotly.graph_objects as go
import copy
import plotly.express as px
import phonotactic_corpora_analysis1 as phc
from legality_principle import LegalitySyllableTokenizer
from legality_principle_gbb import LegalitySyllableTokenizer_gbb

app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY],)

# Styling
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "1rem",
    "padding": "2rem 1rem",
}

BOTTOM_STYLE = {
    "position": "fixed",
    "top": 600,
    "Bottom": 0,
    "margin-left": "18rem",
    "margin-right": "1rem",
    "padding": "2rem 1rem",
}

 

# Title
title = dcc.Markdown("Building Blocks Project",className='bg-primary text-white font-italic', style={"font-size": 30})

#Components

uploader = dcc.Upload(id='upload-data',
        children=html.Div(['Drag & Drop or ',html.A('Select Files')]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '3px',
            'textAlign': 'center',
            'margin': '5px'})


gf = pd.read_table('lang_operations.tsv')

language_options =dcc.RadioItems(id="language",options=gf["Language"].unique(), value=gf["Language"][0],style={'width': '200px'}, )
                                 
filter_options= dcc.Checklist(id="operation",options=gf["Operation"].unique(),value= ["None"],)

filter_input =  dcc.Input(id="name",
                            value="(pa)",
                            className="w-100", )



download_button1 = dbc.Button("Vowel Position Distribution", n_clicks=0,id="btn_csv1", style={'font-size': '1.25em'})


download_button2 = dbc.Button("Root Final Vowel Distribution", n_clicks=0,id="btn_csv2", style={'font-size': '1.25em'})


download_button3 = dbc.Button("Vowel Harmony Analysis", n_clicks=0,id="btn_csv3", style={'font-size': '1.25em'})


download_button4 = dbc.Button("Place of Articulation Analysis", n_clicks=0,id="btn_csv4", style={'font-size': '1.25em'})


file_type= dcc.Dropdown(id="file-type",
                        options=[{"label": "Excel file", "value": "excel"},
                                {"label": "CSV file", "value": "csv"},],
                        placeholder="Choose download file type",
                                 style={'width': '230px'})
vowel_harmony = dcc.Dropdown(['V1V2', 'V2V3', 'V3V4','V4V5', 'V5V6'],value= "V1V2")

poa_options =dcc.RadioItems(id="poa",options=[' By placement', ' Consonant',' Aggregate'], value= " Aggregate",style={'width': '200px'}, )


# Card content
card_content = [
    dbc.CardHeader("'Vowel Position Distribution'"),
    dbc.CardBody(
        [
            html.H5("'Vowel Position Distribution'"),
            html.P(
                "Here you might want to add some statics or further information for your dashboard",
            ),
        ]
    ),
]
## App layout


#Sidebar
sidebar = html.Div([dbc.Row([html.H5('Data Settings',style={"position": "fixed",'margin-top': '10px', 'margin-left': '10px'})],
                            style={"height": "2vh"},),html.Hr(),
dbc.Row([html.Div([html.P('File Upload:', style={'margin-top': '8px', 'margin-bottom': '4px'},className='font-weight-bold'),
                   uploader,dcc.Store(id='raw-file'),
                   html.P('Language:',style={'margin-top': '8px', 'margin-bottom': '4px'},
                           className='font-weight-bold'),language_options,html.Br(),
                    html.P('Corpus Filter Options:',
                           style={'margin-top': '86x', 'margin-bottom': '4px'},
                           className='font-weight-bold'), filter_options, html.Br(),'String removal Filter:',
                           filter_input, dcc.Store(id='table-master'),html.Br(),
                   dbc.Row([html.H5('Analysis Settings',style={'margin-top': '6px', 'margin-left': '10px'})],
                            style={"height": "2vh"}),
                    html.Hr(), html.P('Vowel Harmony Combinations:',
                           style={'margin-top': '16px', 'margin-bottom': '4px'},
                           className='font-weight-bold'), vowel_harmony,html.P('P.O.A. Combinations:',
                           style={'margin-top': '16px', 'margin-bottom': '4px'},
                           className='font-weight-bold'),poa_options])])],
                   style=SIDEBAR_STYLE,)

# Content

        

top_content = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Tabular View', children=[dash_table.DataTable(id='output-data-upload', export_format="csv",filter_action="native", 
        row_deletable=True,sort_action="native",sort_mode="multi",fill_width=False,editable=True,column_selectable="single", page_size= 10,
    style_data={'whiteSpace': 'normal','height': 'auto','lineHeight': '10px'})

        ]),
        dcc.Tab(label='Bar Plot', children=[html.Div(dcc.Graph(
            id='table-paging-with-graph-container',)),



                                            
 ]),
    ])
],style = CONTENT_STYLE)



bottom_content  = html.Div([ 
dbc.Row([dbc.Col([ dbc.Row([dbc.Col([dbc.Card(html.Div([download_button1,html.Br(),html.Br(),
            html.P("Distribution of vowels per syllable positions",),html.Div([html.Hr(),
        dcc.Download(id="download-vtable"),dash_table.DataTable(id='vtable_data',export_format="csv",filter_action="native", 
        row_deletable=True,sort_action="native",sort_mode="multi",fill_width=False,editable=True,column_selectable="single", page_size= 8,style_data={'whiteSpace': 'normal','height': 'auto','lineHeight': '10px'},),])],
        className="text-center text-nowrap my-2 p-2",), color="light")]),
        dbc.Col([dbc.Card(html.Div([download_button2,html.Br(),html.Br(),html.P("Distribution of vowels in root final position",),
                                    html.Div([html.Hr(),
        dash_table.DataTable(id='rtable_data',export_format="csv",filter_action="native", 
        row_deletable=True,sort_action="native",sort_mode="multi",fill_width=False,editable=True,column_selectable="single", page_size= 8,style_data={'whiteSpace': 'normal','height': 'auto','lineHeight': '10px'},),])],
        className="text-center text-nowrap my-2 p-2",), color="light")]),
                            dbc.Col([dbc.Card(html.Div([download_button3,html.Br(),html.Br(),html.P("Distribution of vowel transition from one syllable to another",),
                                                        html.Div([html.Hr(),
        dash_table.DataTable(id='vhartable_data',export_format="csv",filter_action="native", 
        row_deletable=True,sort_action="native",sort_mode="multi",fill_width=False,editable=True,column_selectable="single", page_size= 8,style_data={'whiteSpace': 'normal','height': 'auto','lineHeight': '10px'},),])],
        className="text-center text-nowrap my-2 p-2",), color="light")]),
                            dbc.Col([dbc.Card(html.Div([download_button4,html.Br(),html.Br(),html.P("Distribution of place of articulation (P.O.A.) following a vowel",),
                                                        html.Div([html.Hr(),
        dash_table.DataTable(id='poatable_data',export_format="csv",filter_action="native", 
        row_deletable=True,sort_action="native",sort_mode="multi",fill_width=False,editable=True,column_selectable="single", page_size= 8,style_data={'whiteSpace': 'normal','height': 'auto','lineHeight': '10px'},),])],
        className="text-center text-nowrap my-2 p-2",), color="light")]),
                            
                            ]
                        )
                    ],
                    width=15,
                    style={"margin": "auto"},
                ),
            ])
                           
                           ], style=BOTTOM_STYLE)


# App Layout
app.layout = html.Div(
    [
        dbc.Row([dbc.Col([title], style={"text-align": "center", "margin": "auto"})]),
        sidebar,
        top_content, bottom_content
    ]
)


## Callbacks


@app.callback(Output('raw-file', 'data'),
              Input('upload-data', 'contents'),
              State('upload-data', 'filename'))
def update_output(contents, list_of_names,):
    if contents is None:
        raise PreventUpdate
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
    return df.to_json(date_format='iso', orient='split')

@app.callback(
    Output('output-data-upload','data'),
    Output('output-data-upload', 'columns'),
    Output('table-master','data'),
    Input('raw-file','data'),
    Input(component_id=filter_options, component_property="value"),
    Input(component_id=filter_input, component_property="value"),
    Input(component_id=language_options,component_property="value"),
    prevent_initial_call=True,
)
def update_table(frame, check_value, filt_str, lang_value):
    df = pd.read_json(frame, orient='split')
    head = df['headword'].tolist()
    head1 = df['headword'].tolist()
    tag = df['pos'].tolist()
    mean = df['gloss'].tolist()
    if 'None' in check_value:
        head1=head1
    if 'Verb Segmentation' in check_value:
        ptj_vseg = phc.lightverb_space(head1)
        ptj_incho= phc.ptj_incho_segment(ptj_vseg,tag)
        ptj_vseg1 = phc.ptj_verb_segment(ptj_incho,tag)
        head = phc.ptj_augment_segment(ptj_vseg1,tag)
    if 'Verbal Morphology' in check_value and 'Pitjantjatjara' in lang_value:
        head1 = phc.ptj_verb(head,tag)
    if 'Verbal Morphology' in check_value and 'Warlpiri' in lang_value:
        head1 = phc.wbp_verb(head1,tag)
    if 'String removal' in check_value:
        head1 = phc.remove_items(head1,str(filt_str))
    seg1, seg2 = phc.segment(head)
    if 'Verb compound' in check_value:
        head1 = phc.vcompound(head1,seg1,seg2)
    if 'Reduplication' in check_value:
        head1= phc.redup(head1,seg1,seg2)
    if 'Light Verb' in check_value:
        ptj_light= phc.ptj_verb_comp(head1)
        head1=phc.ptj_verb(ptj_light,tag)
    if lang_value == 'Warumungu':
        vow, vow_count = phc.vowel_count(head1)
        ipa = phc.orth2ipa('wrm',head1)
        syll_ipa = phc.orth2ipa('wrm',vow_count)
        syll_clean = phc.remove_items(syll_ipa, '-')
        syll, syll_u= phc.syllable(syll_clean)
    if lang_value == 'Kaytetye':
        ipa =  df['IPA'].tolist()
        vow = phc.gbb_vowel_count(ipa)
        syll, syll_u= phc.syllable(ipa)
        syll_clean = ipa
    if lang_value == 'Pitjantjatjara':
        vow, vow_count = phc.vowel_count(head1)
        ipa = phc.orth2ipa('ptj',head1)
        syll_ipa = phc.orth2ipa('ptj',vow_count)
        syll_clean = phc.remove_items(syll_ipa, '-')
        syll, syll_u= phc.syllable(syll_clean)
    if lang_value == 'Warlpiri':
        vow, vow_count = phc.vowel_count(head1)
        ipa = phc.orth2ipa('wbp',head1)
        syll_ipa = phc.orth2ipa('wbp',vow_count)  
        syll_clean = phc.remove_items(syll_ipa, '-')
        syll, syll_u= phc.syllable(syll_clean)

    if 'Manual Syllables' in check_value:
        sylla = df['manual_syllable'].tolist()
        dff =phc.prestichframe(head,ipa,syll_clean,tag, mean, vow, sylla, seg1,seg2)        
    else:
        dff =phc.prestichframe(head,ipa,syll_clean,tag, mean, vow, syll, seg1,seg2)
    if 'Independent Word' in check_value:
        dff =phc.stichframe(head,ipa,syll_clean,tag, mean, vow, syll, seg1,seg2)
    if 'Drop duplicates' in check_value:
        dff= phc.drop_dup(dff)

    if 'Drop English Loans' in check_value:
        dff = phc.drop_Eng(dff)

    return dff.to_dict('records'), [{"name": i, "id": i, "deletable": True, "selectable": True} for i in dff.columns], dff.to_json(date_format='iso', orient='split',force_ascii=False)

@app.callback(
    Output('output-data-upload', 'style_data_conditional'),
    Input('output-data-upload', 'selected_columns'),
    prevent_initial_call=True,
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
    
@app.callback(
    Output('table-paging-with-graph-container', "figure"),
    Input('output-data-upload','derived_virtual_data'),
    Input('output-data-upload', "derived_virtual_selected_rows"),
    prevent_initial_call=True,
)
    
def update_graph(rows,derived_virtual_selected_rows):
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []

    dff = df if rows is None else pd.DataFrame(rows)
    colors = ['#7FDBFF' if i in derived_virtual_selected_rows else '#0074D9'
              for i in range(len(dff))]
    if dff.empty:
        fig = px.histogram(dff, x=[])
        return fig
    else:
        fig = px.histogram(dff, x="syllable_count",text_auto =True)
        fig.update_layout(bargap=0.1, font=dict(family="Courier New, monospace", size=14))
        return fig




@app.callback(
    Output('vtable_data','data'),
    Output('vtable_data', 'columns'),
    Input('output-data-upload','derived_virtual_data'),
    Input('output-data-upload', "derived_virtual_selected_rows"),
    Input(component_id=language_options,component_property="value"),
    prevent_initial_call=True,
)
def vowel_position(rows,derived_virtual_selected_rows,lang_value):
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []

    dff = df if rows is None else pd.DataFrame(rows)
    colors = ['#7FDBFF' if i in derived_virtual_selected_rows else '#0074D9'
              for i in range(len(dff))]

    if dff.empty:
        return dff.to_dict('records'), [{"name": i, "id": i, "deletable": False, "selectable": True} for i in dff.columns]
    else: 
        syllables, syllable_len = phc.count_syllables(dff)
        y =phc.count_syllable_position(syllables, syllable_len)
        if 'Kaytetye' in lang_value: 
            vowel_frame = phc.gbb_vowel_distribution(y)
        else:
            vowel_frame = phc.vowel_distribution(y)
        return vowel_frame.to_dict('records'), [{"name": i, "id": i, "deletable": False, "selectable": True} for i in vowel_frame.columns]

@app.callback(
    Output('rtable_data','data'),
    Output('rtable_data', 'columns'),
    Input('output-data-upload','derived_virtual_data'),
    Input('output-data-upload', "derived_virtual_selected_rows"),
    Input(component_id=language_options,component_property="value"),
    prevent_initial_call=True,
)
def rfinal_vowel(rows,derived_virtual_selected_rows,lang_value):
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []

    dff = df if rows is None else pd.DataFrame(rows)
    colors = ['#7FDBFF' if i in derived_virtual_selected_rows else '#0074D9'
              for i in range(len(dff))]
    if dff.empty:
        return dff.to_dict('records'), [{"name": i, "id": i, "deletable": False, "selectable": True} for i in dff.columns]
    else: 
        syllables, syllable_len = phc.count_syllables(dff)
        word = phc.word_length(syllables, syllable_len)
        if 'Kaytetye' in lang_value: 
            root_final_vowel = phc.gbb_count_vowels(word)
        else:
            root_final_vowel = phc.count_vowels(word)
        return root_final_vowel.to_dict('records'),[{"name": i, "id": i, "deletable": False, "selectable": True} for i in root_final_vowel.columns]

@app.callback(
    Output('vhartable_data','data'),
    Output('vhartable_data', 'columns'),
    Input('output-data-upload','derived_virtual_data'),
    Input('output-data-upload', "derived_virtual_selected_rows"),
    Input(vowel_harmony,'value'),
    prevent_initial_call=True,
)
def vowel_harm(rows,derived_virtual_selected_rows,vspec):
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []

    dff = df if rows is None else pd.DataFrame(rows)
    colors = ['#7FDBFF' if i in derived_virtual_selected_rows else '#0074D9'
              for i in range(len(dff))]
    if dff.empty:
        return dff.to_dict('records'), [{"name": i, "id": i, "deletable": False, "selectable": True} for i in dff.columns]
    else:
        syllables, syllable_len = phc.count_syllables(dff)
        syll = phc.clean_syllables(dff)
        syl_matrix = pd.DataFrame(syll, columns=['sy1'])
        syl_matrix[['sy1', 'sy2']] = syl_matrix['sy1'].str.split(',', 1, expand=True).fillna('')  
        if max(syllable_len) == 3:
            syl_matrix[['sy2', 'sy3']] = syl_matrix['sy2'].str.split(',', 1, expand=True).fillna('')
        if max(syllable_len) == 4:
            syl_matrix[['sy2', 'sy3']] = syl_matrix['sy2'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy3', 'sy4']] = syl_matrix['sy3'].str.split(',', 1, expand=True).fillna('')
        if max(syllable_len) == 5:
            syl_matrix[['sy2', 'sy3']] = syl_matrix['sy2'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy3', 'sy4']] = syl_matrix['sy3'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy4', 'sy5']] = syl_matrix['sy4'].str.split(',', 1, expand=True).fillna('')
        if max(syllable_len) == 6:
            syl_matrix[['sy2', 'sy3']] = syl_matrix['sy2'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy3', 'sy4']] = syl_matrix['sy3'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy4', 'sy5']] = syl_matrix['sy4'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy5', 'sy6']] = syl_matrix['sy5'].str.split(',', 1, expand=True).fillna('')
        if max(syllable_len) == 7:
            syl_matrix[['sy2', 'sy3']] = syl_matrix['sy2'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy3', 'sy4']] = syl_matrix['sy3'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy4', 'sy5']] = syl_matrix['sy4'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy5', 'sy6']] = syl_matrix['sy5'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy6', 'sy7']] = syl_matrix['sy6'].str.split(',', 1, expand=True).fillna('')
        if max(syllable_len) == 8:
            syl_matrix[['sy2', 'sy3']] = syl_matrix['sy2'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy3', 'sy4']] = syl_matrix['sy3'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy4', 'sy5']] = syl_matrix['sy4'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy5', 'sy6']] = syl_matrix['sy5'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy6', 'sy7']] = syl_matrix['sy6'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy7', 'sy8']] = syl_matrix['sy7'].str.split(',', 1, expand=True).fillna('')
        if max(syllable_len) == 9:
            syl_matrix[['sy2', 'sy3']] = syl_matrix['sy2'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy3', 'sy4']] = syl_matrix['sy3'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy4', 'sy5']] = syl_matrix['sy4'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy5', 'sy6']] = syl_matrix['sy5'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy6', 'sy7']] = syl_matrix['sy6'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy7', 'sy8']] = syl_matrix['sy7'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy8', 'sy9']] = syl_matrix['sy8'].str.split(',', 1, expand=True).fillna('')
        if max(syllable_len) == 10:
            syl_matrix[['sy2', 'sy3']] = syl_matrix['sy2'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy3', 'sy4']] = syl_matrix['sy3'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy4', 'sy5']] = syl_matrix['sy4'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy5', 'sy6']] = syl_matrix['sy5'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy6', 'sy7']] = syl_matrix['sy6'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy7', 'sy8']] = syl_matrix['sy7'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy8', 'sy9']] = syl_matrix['sy8'].str.split(',', 1, expand=True).fillna('')
            syl_matrix[['sy9', 'sy10']] = syl_matrix['sy9'].str.split(',', 1, expand=True).fillna('')
        vowel_matrix=phc.vowel_matrix(syl_matrix)
        df_out = vowel_matrix.groupby(['sy1', 'sy2']).size().reset_index(name='count')
        if vspec == 'V2V3':
            df_out = vowel_matrix.groupby(['sy2', 'sy3']).size().reset_index(name='count')
        if vspec =='V3V4':
            df_out = vowel_matrix.groupby(['sy3', 'sy4']).size().reset_index(name='count')
        if vspec == 'V4V5':
            df_out = vowel_matrix.groupby(['sy4', 'sy5']).size().reset_index(name='count')
        if vspec == 'V5V6':
            df_out = vowel_matrix.groupby(['sy5', 'sy6']).size().reset_index(name='count')
        return df_out.to_dict('records'),[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df_out.columns]

@app.callback(
    Output('poatable_data','data'),
    Output('poatable_data', 'columns'),
    Input('output-data-upload','derived_virtual_data'),
    Input('output-data-upload', "derived_virtual_selected_rows"),
    Input(component_id=language_options,component_property="value"),
    Input(poa_options,'value'),
    prevent_initial_call=True,
)
        
def poa_dist(rows,derived_virtual_selected_rows, lang,poa_filter):
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []

    dff = df if rows is None else pd.DataFrame(rows)
    colors = ['#7FDBFF' if i in derived_virtual_selected_rows else '#0074D9'
              for i in range(len(dff))]
    if dff.empty:
        return dff.to_dict('records'), [{"name": i, "id": i, "deletable": False, "selectable": True} for i in dff.columns]
    else:
        syll_clean= dff['OS'].tolist()
        word_tmp = phc.word_template(lang,syll_clean)
        onset, coda = phc.VC_clusters(syll_clean,word_tmp)
        onset_poa =phc.poa_labeller(onset)
        coda_poa =phc.poa_labeller(coda)
        onset_lab = list(len(onset)*['onset'])
        coda_lab = list(len(coda)*['coda'])
        onsetV, onsetC = phc.VC_spliter(onset)
        codaV, codaC = phc.VC_spliter(coda)
        bf = pd.DataFrame(list(zip(onset,onsetV,onsetC,onset_lab, onset_poa)),columns=['VC','V','C','placement','poa'])
        gf = pd.DataFrame(list(zip(coda,codaV,codaC,coda_lab, coda_poa)),columns=['VC','V','C','placement','poa'])
        ff = pd.concat([bf,gf])
        if poa_filter == ' Aggregate':
            poatab= ff.groupby(['V', 'poa']).size().reset_index(name='count')
        if poa_filter == ' By placement':
            poatab= ff.groupby(['V', 'poa', 'placement']).size().reset_index(name='count')
        if poa_filter == ' Consonant':
            poatab= ff.groupby(['V', 'C']).size().reset_index(name='count')
        return poatab.to_dict('records'),[{"name": i, "id": i, "deletable": False, "selectable": True} for i in poatab.columns]

@app.callback(
    Output("language", "options"),
    Input("operation", "value"),
)
def chained_callback_language(operation):
    gff = copy.deepcopy(gf)
    if operation is not None:
        gff = gff.query("Operation == @operation")
    return sorted(gff["Language"].unique())

@app.callback(
    Output("operation", "options"),
    Input("language", "value"),
)
def chained_callback_operation(language):
    gff = copy.deepcopy(gf)
    if language is not None:
        gff = gff.query("Language == @language")
    return sorted(gff["Operation"].unique())

if __name__ == "__main__":
    app.run_server(debug=True, external ='tab', port =1200)

In [3]:
!pip install print-versions # works for python >= 3.8

    opencv-python (>=3.) ; extra == 'all'
                  ~~~~^
    PyYAML (>=5.1.*)
            ~~~~~~^


In [6]:
from print_versions import print_versions

import numpy as np
from pandas import DataFrame

print_versions(globals())

pip==24.2
numpy==1.24.3
pandas==1.5.3
dash==2.18.1
dash_bootstrap_components==1.6.0
dash.dash_table==5.2.12
dash.dcc==2.14.2
dash.html==2.0.19
